# Notebook

In [258]:
import math
import yaml

## Book Keeping

In [259]:
MODEL = "PE1R_112_M4"
VERBOSITY = 1

In [260]:
SWITCHING_FREQUENCY = 5000
BUS_VOLTAGE = 500
PEAK_CARRIER_VOLTAGE = 2
GEARBOX_GAIN = 10
LOAD_MOMENT_OF_INERTIA = 120
LOAD_FRICTION_COEFFICIENT = 31.8
CURRENT_SENSOR_GAIN = 1 / 60
CURRENT_RESPONSE_TIME = 2e-3
CURRENT_OVERSHOOT = 7
SPEED_SENSOR_GAIN =  1 / 40
SPEED_RESPONSE_TIME = 20e-3
SPEED_OVERSHOOT = 5
POSITION_SENSOR_GAIN =  1 / 330
POSITION_RESPONSE_TIME = 200e-3
POSITION_OVERSHOOT = 4

In [261]:
with open("../reports/" + MODEL + "_parameters.yaml", "r") as file:
    parameters = yaml.safe_load(file)

In [262]:
Kc = 2 * BUS_VOLTAGE / PEAK_CARRIER_VOLTAGE
Ts = 1 / SWITCHING_FREQUENCY

if VERBOSITY:
    print("Kc :: {0}".format(Kc))

Kc :: 500.0


In [263]:
Rs = parameters["provided_parameters"]["Rs"]
Ld = parameters["estimated_parameters"]["Lq"]

tau_maid = Ld / Rs
Kmaid = (Kc * CURRENT_SENSOR_GAIN) / Rs
tau_mfid = CURRENT_RESPONSE_TIME / (3 * 10)
zeta_id = -math.log(CURRENT_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(CURRENT_OVERSHOOT / 100)**2)
Kpid = ((2 * tau_maid) / tau_mfid - 1) / Kmaid
Tiid = (tau_mfid**2 * zeta_id**2 * Kmaid) / tau_maid
tau_zid = Kpid * Tiid

if VERBOSITY:
    print("Kpid :: {0}".format(Kpid))
    print("Tiid :: {0}".format(Tiid))
    print("tau_zid :: {0}".format(tau_zid))

Kpid :: 5.12856
Tiid :: 1.0849183731345302e-05
tau_zid :: 5.5640689717228266e-05


In [264]:
Lq = parameters["estimated_parameters"]["Lq"]

tau_maiq = Lq / Rs
Kmaiq = (Kc * CURRENT_SENSOR_GAIN) / Rs
tau_mfiq = CURRENT_RESPONSE_TIME / 3
zeta_iq = -math.log(CURRENT_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(CURRENT_OVERSHOOT / 100)**2)
Kpiq = ((2 * tau_maiq) / tau_mfiq - 1) / Kmaiq
Tiiq = (tau_mfiq**2 * zeta_iq**2 * Kmaiq) / tau_maiq
tau_ziq = Kpiq * Tiiq

if VERBOSITY:
    print("Kpiq :: {0}".format(Kpiq))
    print("Tiiq :: {0}".format(Tiiq))
    print("tau_ziq :: {0}".format(tau_ziq))

Kpiq :: 0.5115600000000001
Tiiq :: 0.00108491837313453
tau_ziq :: 0.0005550008429607003


In [265]:
Br = parameters["estimated_parameters"]["Br"]
Jr = parameters["provided_parameters"]["Jr"]
Ke = parameters["estimated_parameters"]["Ke"]
Kt = parameters["estimated_parameters"]["Kt"]

Jeq = Jr + LOAD_MOMENT_OF_INERTIA / GEARBOX_GAIN**2
Beq = Br + LOAD_FRICTION_COEFFICIENT / GEARBOX_GAIN**2
Jti = (Ke * Kt * Tiiq) / (Kc * CURRENT_SENSOR_GAIN)
Ksv = SPEED_SENSOR_GAIN * (30 / math.pi) 

tau_mav = (Jeq + Jti) / Beq
Kmav = (Kt * Ksv) / (Beq * GEARBOX_GAIN * CURRENT_SENSOR_GAIN)
tau_mfv = SPEED_RESPONSE_TIME / 3
zeta_v = -math.log(SPEED_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(SPEED_OVERSHOOT / 100)**2)
Kpv = ((2 * tau_mav) / tau_mfv - 1) / Kmav
Tiv = (tau_mfv**2 * zeta_v**2 * Kmav) / tau_mav
tau_zv = Kpv * Tiv

if VERBOSITY:
    print("Kpv :: {0}".format(Kpv))
    print("Tiv :: {0}".format(Tiv))
    print("tau_zv :: {0}".format(tau_zv))


Kpv :: 185.8920889046477
Tiv :: 3.413527534253094e-05
tau_zv :: 0.0063454776387583895


In [266]:
Ksp = POSITION_SENSOR_GAIN * (180 / math.pi)
Kmap = Ksp / Ksv
tau_mfp = POSITION_RESPONSE_TIME / 3
Kpv = 1 / (Kmap * tau_mfp)

if VERBOSITY:
    print("Kpv :: {0}".format(Kpv))

Kpv :: 20.625000000000004
